In [2]:
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from helpers import snowflake_config

In [3]:
credentials = snowflake_config.credenciales()

motor = create_engine(URL(
    account = credentials['account'],
    user = credentials['user'],
    password = credentials['password'],
    database = credentials['database'],
    schema = credentials['schema'],
    warehouse = credentials['warehouse'],
    role = credentials['role']
))

In [4]:
consulta = 'select * from ordenes'

crudo_ordenes = pd.read_sql(consulta, con=motor)

In [5]:

clean_ordenes = crudo_ordenes.drop_duplicates().reset_index(drop=True)

In [6]:
clean_ordenes

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1515936,183418,11,6,13,30.0
1,1690866,163593,5,5,12,9.0
2,1454967,39980,4,5,19,2.0
3,1768857,82516,56,0,20,10.0
4,3007858,196724,2,4,12,17.0
...,...,...,...,...,...,...
478947,3210681,5617,5,1,14,7.0
478948,3270802,112087,2,3,13,6.0
478949,885349,82944,16,2,11,6.0
478950,216274,4391,3,3,8,8.0


In [7]:
clean_ordenes.shape

(478952, 6)

In [8]:
clean_ordenes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478952 entries, 0 to 478951
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                478952 non-null  int64  
 1   user_id                 478952 non-null  int64  
 2   order_number            478952 non-null  int64  
 3   order_dow               478952 non-null  int64  
 4   order_hour_of_day       478952 non-null  int64  
 5   days_since_prior_order  450135 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 21.9 MB


In [9]:
clean_ordenes['days_since_prior_order'] = clean_ordenes['days_since_prior_order'].fillna(-1)

In [10]:
consulta = 'select * from productos'
crudo_productos = pd.read_sql(consulta, con=motor)

In [11]:
clean_productos = crudo_productos.fillna('Producto Desconocido')

In [12]:
clean_productos['product_name_upper'] = clean_productos['product_name'].str.upper()

In [13]:
consulta = 'select * from ordenes_productos'
crudo_ordenes_productos = pd.read_sql(consulta, con=motor)

In [14]:
clean_ordenes_productos = crudo_ordenes_productos.fillna(-1)

In [15]:
query = 'select * from pasillos'
crudo_pasillos = pd.read_sql(query, con=motor)

In [16]:
crudo_pasillos['nombre_pasillo'] = crudo_pasillos['aisle'].str.upper()
clean_pasillos = crudo_pasillos

In [17]:
consulta = 'select * from departamentos'
crudo_departamentos = pd.read_sql(consulta, con=motor)

In [18]:
crudo_departamentos['nombre_departamento'] = crudo_departamentos['department'].str.upper()

In [19]:
clean_departamentos = crudo_departamentos

In [20]:
clean_ordenes.to_csv('../datasets/silver/ordenes.csv')
clean_ordenes_productos.to_csv('../datasets/silver/ordenes_productos.csv')
clean_productos.to_csv('../datasets/silver/productos.csv')
clean_departamentos.to_csv('../datasets/silver/departamentos.csv')
clean_pasillos.to_csv('../datasets/silver/pasillos.csv')

In [21]:
dim_productos = clean_productos.merge(clean_pasillos, how='inner', on='aisle_id')
dim_productos = dim_productos.merge(clean_departamentos, how='inner', on='department_id')

In [22]:
fct_ordenes = clean_ordenes.merge(clean_ordenes_productos, how='inner', on='order_id')

In [23]:
fct_ordenes

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1515936,183418,11,6,13,30.0,22035,13.0,0
1,1515936,183418,11,6,13,30.0,21903,6.0,1
2,1515936,183418,11,6,13,30.0,19048,1.0,1
3,1515936,183418,11,6,13,30.0,47766,3.0,1
4,1515936,183418,11,6,13,30.0,45066,9.0,0
...,...,...,...,...,...,...,...,...,...
4545002,2071924,1730,18,1,14,15.0,45037,15.0,0
4545003,2071924,1730,18,1,14,15.0,47157,9.0,1
4545004,2071924,1730,18,1,14,15.0,43772,3.0,1
4545005,2071924,1730,18,1,14,15.0,47626,5.0,1


In [24]:
dim_productos.to_csv('../datasets/gold/dim_products.csv', index=False)
fct_ordenes.to_csv('../datasets/gold/fct_ordenes.csv', index=False)
